# Class 17

## libraries

In [2]:
import sqlite3
import random
import hashlib
import time

## Exercise 1: Implement emoji support in passwords using UTF-8 encoding. If your command line doesn't support emojis, consider a web-based solution.

In [3]:
def connect ():
  try:
    conn = sqlite3.connect('users.db', timeout=100)  # Set a timeout to avoid locking
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users
               (username TEXT UNIQUE, password TEXT, salt TEXT)''')
    conn.commit()
    return conn
  except sqlite3.OperationalError as e:
    print(f"Database error: {e}")
    return None

def hash_password_with_salt (password, salt=random.randint(0, 100)):
  salt = str(salt)
  password += str(salt)
  password = password.encode('utf-8')
  hashed_password = hashlib.sha256(password).hexdigest()
  return hashed_password, salt

def register (username, password):
  conn = connect()
  password_hash, salt = hash_password_with_salt(password)
  c = conn.cursor()
  try:
    c.execute("INSERT INTO users VALUES (?,?,?)", (username, password_hash, salt))
    conn.commit()
    print(f"{username} is registered completely")
  except sqlite3.IntegrityError as e:
    print(f"Database error: {e}")
  finally:
    close_connect(conn)

def login (username, password):
  conn = connect()
  c = conn.cursor()
  c.execute("SELECT * FROM users WHERE username=?", (username,))
  user = c.fetchone()
  close_connect(conn)
  if user:
    salt = user[2]
    hashed_password, _ = hash_password_with_salt(password, salt)
    if hashed_password == user[1]:
      return True
  return False

def close_connect(conn):
  if conn:
    conn.close()

In [4]:
# test registration
register('user1', 'password1✅')
print(login('user1', 'password1✅')) # True
print(login('user1', 'password1')) # False

Database error: UNIQUE constraint failed: users.username
True
False


## Exercise 2: Write two functions for user registration and login using a local file as a password.

In [5]:
def register_user(file_path):
    with open(file_path, 'rb') as f:
        file_content = f.read()
    password_hash = hashlib.sha256(file_content).hexdigest()
    return password_hash

def login_user(file_path, registered_hash):
    with open(file_path, 'rb') as f:
        file_content = f.read()
    login_hash = hashlib.sha256(file_content).hexdigest()
    return login_hash == registered_hash

def output(is_authenticated):
    print("Login successful!" if is_authenticated else "Login failed!")
# Demonstration
registered_file = 'registered_file.txt'
registered_hash = register_user(registered_file)

output(login_user(file_path="login_file.txt", registered_hash=registered_hash))
output(login_user(file_path="wrong_file.txt", registered_hash=registered_hash))


Login successful!
Login failed!


## Exercise 3: Implement a one-time password (OTP) generator using a hashing function, including scripts for registration and login.

In [6]:
class OTPSystem:
    def __init__(self, password, salt):
        self.password = password
        self.salt = salt
        self.hsp = self.hash_password()

    def hash_password(self):
        return hashlib.sha256((self.password + self.salt).encode()).hexdigest()

    def update_hsp(self):
        current_time = str(int(time.time()))
        self.hsp = hashlib.sha256((self.hsp + current_time).encode()).hexdigest()

    def generate_otp(self):
        return self.hsp[-6:]

    def verify_otp(self, otp):
        return otp == self.generate_otp()

# Demonstration
otp_system = OTPSystem(password="mypassword", salt="mysalt")
otp_system.update_hsp()
otp = otp_system.generate_otp()
print(f"Generated OTP: {otp}")

# Verify OTP
user_otp = input("Enter OTP: ")
if otp_system.verify_otp(user_otp):
    print("Login successful!")
else:
    print("Login failed!")


Generated OTP: 1f9d11
Login failed!


## Exercise 4: Evaluate the performance of a biometric authentication system by calculating the False Accept Rate (FAR) and False Reject Rate (FRR) based on given data for Alice and Eve.

In [7]:
# Data for Alice (genuine user) and Eve (attacker)
alice_data = [
    0.27, 0.079, 0.27, 0.36, 0.25, 0.3, 0.27, 0.17, 0.23, 0.12, 0.29, 0.085, 0.093, 0.12, 0, 0.34, 0.23, 0.12,
    0.34, 0.029, 0.19, 0.18, 0.23, 0.23, 0.11, 0.2, 0.18, 0.26, 0.31, 0.31, 0.11, 0.21, 0.079, 0.089, 0.2, 0.35,
    0.12, 0.24, 0.18, 0.31, 0.091, 0.2, 0.26, 0.31, 0.35, 0.21, 0.051, 0.13, 0.094, 0.44, 0.14, 0.27, 0.18, 0.29,
    0.12, 0.06, 0.058, 0.25, 0.18, 0.18, 0.34, 0.23, 0.22, 0.36, 0.12, 0.27, 0.28, 0.18, 0.22, 0.083, 0.085, 0.21,
    0.27, 0.46, 0.13, 0.22, 0.19, 0.0067, 0.16, 0.021, 0.28, 0.11, 0.21, 0.15, 0.23, 0.14, 0.25, 0.27, 0.37, 0.18,
    0, 0.12, 0.34, 0.093, 0.3, 0.21, 0.34, 0.0039, 0.18, 0.079
]
eve_data = [
    1.2, 0.77, 0.88, 0.39, 0.51, 0.55, 0.82, 0.54, 0.74, 0.19, 0.53, 0.44, 0.28, 0.7, 0.66, 0.61, 0.33, 0.83,
    0.67, 0.54, 0.6, 0.55, 0.25, 0.54, 0.43, 0.4, 0.37, 0.49, 0.2, 0.79, 0.7, 0.6, 0.59, 0.44, 0.8, 0.57, 0.46,
    0.87, 0.56, 0.48, 0.54, 0.43, 0.38, 1.1, 0.93, 0.66, 0.35, 0.43, 0.56, 0.76, 0.33, 0.13, 0.31, 0.67, 0.68,
    0.69, 0.57, 0.64, 0.5, 0.77, 0.33, 0.69, 0.43, 0.53, 0.71, 0.81, 0.38, 0.85, 0.73, 0.59, 0.56, 0.56, 0.54,
    0.6, 0.61, 0.77, 0.91, 0.69, 0.56, 0.73, 0.64, 0.39, 0.79, 0.66, 0.63, 0.7, 0.65, 0.41, 0.57, 0.57, 0.49,
    0.94, 0.42, 0.5, 0.46, 0.37, 0.56, 0.55, 0.91, 0.55
]

def print_stats(thresholds):
    for threshold in thresholds:
        frr = len([x for x in alice_data if x >= threshold]) / len(alice_data) * 100
        far = len([x for x in eve_data if x < threshold]) / len(eve_data) * 100
        print(f"Threshold: {threshold}, FAR: {far:.2f}%, FRR: {frr:.2f}%")


In [8]:
# recommend one
thresholds = [0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55]
print_stats(thresholds)

Threshold: 0.25, FAR: 3.00%, FRR: 34.00%
Threshold: 0.3, FAR: 5.00%, FRR: 18.00%
Threshold: 0.35, FAR: 9.00%, FRR: 7.00%
Threshold: 0.4, FAR: 16.00%, FRR: 2.00%
Threshold: 0.45, FAR: 25.00%, FRR: 1.00%
Threshold: 0.5, FAR: 30.00%, FRR: 0.00%
Threshold: 0.55, FAR: 40.00%, FRR: 0.00%


In [9]:
thresholds = [0.20, 0.27, 0.35, 0.42, 0.50, 0.57, 0.65]
print_stats(thresholds)

Threshold: 0.2, FAR: 2.00%, FRR: 52.00%
Threshold: 0.27, FAR: 4.00%, FRR: 29.00%
Threshold: 0.35, FAR: 9.00%, FRR: 7.00%
Threshold: 0.42, FAR: 18.00%, FRR: 2.00%
Threshold: 0.5, FAR: 30.00%, FRR: 0.00%
Threshold: 0.57, FAR: 50.00%, FRR: 0.00%
Threshold: 0.65, FAR: 64.00%, FRR: 0.00%


In [10]:
thresholds = [0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40]
print_stats(thresholds)

Threshold: 0.1, FAR: 0.00%, FRR: 80.00%
Threshold: 0.15, FAR: 1.00%, FRR: 66.00%
Threshold: 0.2, FAR: 2.00%, FRR: 52.00%
Threshold: 0.25, FAR: 3.00%, FRR: 34.00%
Threshold: 0.3, FAR: 5.00%, FRR: 18.00%
Threshold: 0.35, FAR: 9.00%, FRR: 7.00%
Threshold: 0.4, FAR: 16.00%, FRR: 2.00%


In [11]:
thresholds = [0.11, 0.22, 0.33, 0.44, 0.55, 0.66, 0.77]
print_stats(thresholds)

Threshold: 0.11, FAR: 0.00%, FRR: 80.00%
Threshold: 0.22, FAR: 3.00%, FRR: 44.00%
Threshold: 0.33, FAR: 6.00%, FRR: 12.00%
Threshold: 0.44, FAR: 23.00%, FRR: 2.00%
Threshold: 0.55, FAR: 40.00%, FRR: 0.00%
Threshold: 0.66, FAR: 65.00%, FRR: 0.00%
Threshold: 0.77, FAR: 82.00%, FRR: 0.00%


In [12]:
thresholds = [0.23, 0.34, 0.45, 0.56, 0.67, 0.78, 0.89]
print_stats(thresholds)

Threshold: 0.23, FAR: 3.00%, FRR: 41.00%
Threshold: 0.34, FAR: 9.00%, FRR: 12.00%
Threshold: 0.45, FAR: 25.00%, FRR: 1.00%
Threshold: 0.56, FAR: 44.00%, FRR: 0.00%
Threshold: 0.67, FAR: 68.00%, FRR: 0.00%
Threshold: 0.78, FAR: 85.00%, FRR: 0.00%
Threshold: 0.89, FAR: 94.00%, FRR: 0.00%


In [13]:
thresholds = [0.46, 0.50, 0.54, 0.59, 0.63, 0.67, 0.71]
print_stats(thresholds)

Threshold: 0.46, FAR: 25.00%, FRR: 1.00%
Threshold: 0.5, FAR: 30.00%, FRR: 0.00%
Threshold: 0.54, FAR: 35.00%, FRR: 0.00%
Threshold: 0.59, FAR: 54.00%, FRR: 0.00%
Threshold: 0.63, FAR: 61.00%, FRR: 0.00%
Threshold: 0.67, FAR: 68.00%, FRR: 0.00%
Threshold: 0.71, FAR: 77.00%, FRR: 0.00%
